Useless

In [1]:
# % cd drive/MyDrive/Shiyuan/
# % ls

Starts Here

## Data
### 2016 Eora26

## Export and Import
- TID: Total Intermediate demand;
- TFD: Total Final demand;
- HID: Home Intermediate demand;
- FID: Foreign Intermediate demand;
- HFD: Home Final demand;
- FFD: Foreign Final demand.

#### Export calculation method:
1. TID: Intermediate matrix sum by rows; TFD: Final demand matrix sum by rows.
2. Trade mask (a matrix with the same size as Intermediate matrix or Final demand matrix, which has zeros for domestic flow and 1 for other flows); Multiply trade mask by Intermediate matrix or Final demand matrix elementwisely to derive masked Intermediate matrix or Final demand matrix; A masked matrix only contains export flows.
3. FID: Masked Intermediate matrix sum by rows; FFD: Masked Final demand matrix sum by rows.
4. HID = TID - FID; HFD = TFD - FFD

#### Import calculation method:
- Import into industries: Masked Intermediate matrix sum by columns;
- Import by final demand: Masked Final demand matrix sum by columns.

#### Check correctness:
- **Gross input must be equal gross output (xin = xout).**

In [3]:
import pandas as pd
import numpy as np
import json
import copy

In [4]:
# Read files
## Final demand matrix
f = open('C:/Users/15311/Desktop/Global_PM2.5/EURO/EURO26/Eora26_2016_bp_FD.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")
    newData[i][-1] = newData[i][-1][:-1]
FD_df = pd.DataFrame(newData)
f.close()

f = open('C:/Users/15311/Desktop/Global_PM2.5/EURO/EURO26/labels_T.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")[:-1]
LABELS_df = pd.DataFrame(newData)
f.close()

## Intermediate transaction matrix
f = open('C:/Users/15311/Desktop/Global_PM2.5/EURO/EURO26/Eora26_2016_bp_T.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")
    newData[i][-1] = newData[i][-1][:-1]
T_df = pd.DataFrame(newData)
f.close()

In [4]:
# From str to float : T_df, FD_df
T_df = T_df.astype('float')
FD_df = FD_df.astype('float');

In [5]:
FD_df

,0,1,2,3,4,5,6,7,8,9,...,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139
0,2.441240e+06,21191.40000,4170.41000,17477.90000,9921.32000,50.29780,1.369080,1.338850,2.235760,2.504540,...,1.067240,1.420560,0.030502,1.050150,0.0,0.0,0.0,0.0,0.0,0.0
1,9.756520e+04,526.74900,2603.48000,7.44061,793.11800,7.44061,0.210492,0.210492,0.244429,0.255776,...,0.709638,0.598005,0.020051,0.598005,0.0,0.0,0.0,0.0,0.0,0.0
2,1.923700e+04,18.99120,334.61100,548.32100,1452.70000,1.78847,0.156236,0.156236,0.246782,0.215100,...,0.571534,0.398656,0.020445,0.398656,0.0,0.0,0.0,0.0,0.0,0.0
3,8.358320e+05,11606.10000,1.04009,0.44913,1812.68000,0.44913,0.056894,0.056894,0.080191,0.070835,...,0.398171,0.280365,0.015234,0.280365,0.0,0.0,0.0,0.0,0.0,0.0
4,1.393990e+05,1849.85000,28.03080,5638.92000,329.13900,13.14740,0.097453,0.097453,0.173592,0.149557,...,0.425890,0.348862,0.022607,0.348862,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,6.406050e+00,6.40605,7.61509,7.99647,3.62230,7.99647,5.176600,5.176600,5.871570,6.588000,...,301040.000000,71759.100000,0.000000,71759.100000,0.0,0.0,0.0,0.0,0.0,0.0
4911,2.969850e+00,2.96985,3.40420,3.24971,1.71085,3.24971,1.817520,1.817520,1.933540,2.008380,...,109792.000000,21763.100000,0.000000,21763.100000,0.0,0.0,0.0,0.0,0.0,0.0
4912,7.513970e+00,7.51397,8.74698,8.27024,3.83962,8.27024,5.790060,5.790060,6.324450,6.546780,...,117676.000000,22373.000000,0.000000,22373.000000,0.0,0.0,0.0,0.0,0.0,0.0
4913,1.449490e+01,14.49490,17.07700,15.95110,6.92056,15.95110,11.069300,11.069300,12.226400,12.927700,...,134683.000000,25011.100000,0.000000,25011.100000,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Extact names of countries
LABELS_df.columns = ["Country","c","I","Industries"]
Export = pd.DataFrame(LABELS_df,columns=["Country","Industries"])
Countries = list(LABELS_df["Country"].unique())

In [7]:
file=open("Countries.json",'w')
json.dump(Countries,file)
file.close()

### Export

In [8]:
# Calculate TID and TFD
Export["TID"] = T_df.sum(axis=1) # Total intermediate demand
Export["TFD"] = FD_df.sum(axis=1) # Total final demand

In [9]:
# Calculate trade mask matrix
T_df1 = copy.copy(T_df)
FD_df1 = copy.copy(FD_df)
Masked_T = T_df1.to_numpy()
Masked_FD = FD_df1.to_numpy()

In [10]:
Masked_FD.shape,Masked_T.shape

((4915, 1140), (4915, 4915))

In [11]:
# A masked matrix only contains export flows: zeros for domestic flow.
for i in range(189):
    Masked_T[i*26:(i+1)*26,i*26:(i+1)*26] = 0
    Masked_FD[i*26:(i+1)*26,i*6:(i+1)*6] = 0
Masked_T[-1,-1] = 0
Masked_FD[-1,-6:] = 0

Masked_T_df = pd.DataFrame(Masked_T)
Masked_FD_df = pd.DataFrame(Masked_FD)

In [12]:
# Calculate FID and FFD
Export["FID"] = Masked_T_df.sum(axis=1) # Foreign intermediate demand
Export["FFD"] = Masked_FD_df.sum(axis=1) # Foreign final demand

In [13]:
Export

,Country,Industries,TID,TFD,FID,FFD
0,AFG,Agriculture,1.538278e+06,2.982180e+06,1.118980e+06,488129.032924
1,AFG,Fishing,8.205439e+04,1.021976e+05,3.289848e+04,694.220139
2,AFG,Mining and Quarrying,2.571404e+05,2.229562e+04,4.084204e+04,702.204809
3,AFG,Food & Beverages,4.822598e+05,8.496691e+05,6.116602e+03,416.376645
4,AFG,Textiles and Wearing Apparel,1.526769e+05,1.494909e+05,2.357707e+04,2232.773464
...,...,...,...,...,...,...
4910,ZWE,"Education, Health and Other Services",4.055454e+05,9.167422e+05,4.053208e+05,11057.979058
4911,ZWE,Private Households,5.459519e+04,3.839821e+05,5.458529e+04,2463.884800
4912,ZWE,Others,1.879805e+05,3.851717e+05,1.879705e+05,7405.652550
4913,ZWE,Re-export & Re-import,2.629224e+05,4.401848e+05,2.629099e+05,14487.588642


In [14]:
# Calculate HID and HFD
Export["HID"] = Export["TID"] - Export["FID"] # Home intermediate demand
Export["HFD"] = Export["TFD"] - Export["FFD"] # Home final demand

In [15]:
# Gross output and Export
Export["Gross_output"] = Export["TID"] + Export["TFD"]
Export["Tol_export"] = Export["FID"] + Export["FFD"]

In [16]:
# Calculate total export of each country in a dict.
Tol_export = dict(Export.groupby(by=["Country"])["Tol_export"].sum())

In [17]:
# Simple check
Export[Export["Country"] == "ROW"]["Tol_export"].sum() == Tol_export["ROW"]

True

In [18]:
Tol_export["PSE"]

2570872.5058461623

In [19]:
Export[Export['TFD'] <0]

,Country,Industries,TID,TFD,FID,FFD,HID,HFD,Gross_output,Tol_export
28,ALB,Mining and Quarrying,4.020272e+05,-5.235642e+03,7.079316e+04,4.997713e+03,3.312340e+05,-1.023335e+04,3.967916e+05,7.579087e+04
37,ALB,Recycling,3.363908e+04,-3.444279e+03,2.284055e+04,5.118659e+03,1.079854e+04,-8.562937e+03,3.019480e+04,2.795921e+04
444,BEL,Mining and Quarrying,7.120170e+06,-4.346013e+04,6.444989e+06,4.453414e+04,6.751805e+05,-8.799427e+04,7.076709e+06,6.489523e+06
548,BTN,Mining and Quarrying,7.349024e+04,-3.148992e+02,2.374698e+04,6.673042e+02,4.974326e+04,-9.822035e+02,7.317535e+04,2.441429e+04
553,BTN,Metal Products,1.063308e+05,-3.061443e+03,9.904199e+04,2.253432e+03,7.288777e+03,-5.314875e+03,1.032693e+05,1.012954e+05
...,...,...,...,...,...,...,...,...,...,...
4349,THA,Metal Products,3.658807e+07,-7.853055e+06,1.219864e+07,1.173923e+06,2.438943e+07,-9.026978e+06,2.873502e+07,1.337257e+07
4474,TUR,Mining and Quarrying,9.818546e+06,-9.806123e+05,5.022136e+06,4.945494e+04,4.796410e+06,-1.030067e+06,8.837933e+06,5.071591e+06
4680,USA,Agriculture,3.931922e+08,-1.794295e+07,4.143490e+07,1.826050e+07,3.517573e+08,-3.620345e+07,3.752492e+08,5.969539e+07
4682,USA,Mining and Quarrying,2.889991e+08,-7.527387e+07,2.634329e+07,5.771311e+05,2.626559e+08,-7.585100e+07,2.137253e+08,2.692042e+07


In [20]:
Export[Export['Country'] == 'CHN']

,Country,Industries,TID,TFD,FID,FFD,HID,HFD,Gross_output,Tol_export
1014,CHN,Agriculture,8.085453e+08,6.108391e+08,1.746110e+07,4.733246e+06,7.910842e+08,6.061059e+08,1.419384e+09,2.219435e+07
1015,CHN,Fishing,7.753419e+07,7.787805e+07,1.281784e+06,5.329243e+05,7.625241e+07,7.734512e+07,1.554122e+08,1.814708e+06
1016,CHN,Mining and Quarrying,8.063944e+08,3.465302e+07,1.883950e+07,3.366129e+05,7.875549e+08,3.431640e+07,8.410474e+08,1.917611e+07
1017,CHN,Food & Beverages,6.483990e+08,7.657731e+08,2.550463e+07,2.358296e+07,6.228944e+08,7.421902e+08,1.414172e+09,4.908760e+07
1018,CHN,Textiles and Wearing Apparel,8.092089e+08,4.478153e+08,1.322974e+08,1.953402e+08,6.769116e+08,2.524750e+08,1.257024e+09,3.276376e+08
1019,CHN,Wood and Paper,5.515578e+08,6.553864e+07,3.556917e+07,1.456936e+07,5.159886e+08,5.096928e+07,6.170964e+08,5.013853e+07
1020,CHN,"Petroleum, Chemical and Non-Metallic Mineral P...",3.046699e+09,2.706942e+08,2.382753e+08,4.065909e+07,2.808424e+09,2.300351e+08,3.317393e+09,2.789343e+08
1021,CHN,Metal Products,2.289518e+09,1.061600e+08,1.594627e+08,1.885863e+07,2.130055e+09,8.730135e+07,2.395678e+09,1.783213e+08
1022,CHN,Electrical and Machinery,2.315541e+09,1.553307e+09,4.151334e+08,2.972567e+08,1.900408e+09,1.256050e+09,3.868848e+09,7.123900e+08
1023,CHN,Transport Equipment,7.005294e+08,4.610633e+08,2.769271e+07,3.066634e+07,6.728367e+08,4.303970e+08,1.161593e+09,5.835904e+07


In [21]:
Export.to_csv("Export.csv",index=False) 

### Import

In [22]:
Import_ind = Masked_T_df.sum(axis=0) # Import into industries
Import_fd = Masked_FD_df.sum(axis=0) # Import by final demand

In [23]:
Import_ind.shape,Import_fd.shape

((4915,), (1140,))

In [24]:
Tol_ind = {}
for i in range(189):
    Tol_ind[Countries[i]] = sum(Import_ind[i*26:(i+1)*26])
Tol_ind["ROW"] = Import_ind[len(Import_ind)-1]

In [25]:
Tol_fd = {}
for i in range(190):
    Tol_fd[Countries[i]] = sum(Import_fd[i*6:(i+1)*6])

In [26]:
Tol_import = {}
for i in Countries:
    Tol_import[i] = Tol_fd[i] + Tol_ind[i]

### Check correctness

In [27]:
ratio_con = {}
for i in Countries:
    ratio_con[i] = Tol_export[i]/Tol_import[i]

In [28]:
ratio_con

{'AFG': 0.44150258142631177,
 'ALB': 0.6366334378643103,
 'DZA': 0.5919337025011444,
 'AND': 1.027806531380985,
 'AGO': 1.1071070444655482,
 'ATG': 0.8055992061664792,
 'ARG': 0.9097106648050483,
 'ARM': 0.8018956927179812,
 'ABW': 1.035773696675649,
 'AUS': 1.0674457305024094,
 'AUT': 1.0716656087101748,
 'AZE': 1.0431351430252105,
 'BHS': 1.0528936668494941,
 'BHR': 1.1379030836948272,
 'BGD': 0.8030944643331768,
 'BRB': 0.9091987708134041,
 'BLR': 1.0529320732130536,
 'BEL': 1.0093811225324127,
 'BLZ': 0.8260850044829524,
 'BEN': 0.7387695286665751,
 'BMU': 1.6956147845731258,
 'BTN': 0.6121774035987021,
 'BOL': 0.6984704355616349,
 'BIH': 0.6623836176436309,
 'BWA': 1.1522747877745596,
 'BRA': 0.9958756910820384,
 'VGB': 1.483608504877884,
 'BRN': 1.4194712182666405,
 'BGR': 1.1297677122750875,
 'BFA': 0.9160481621662786,
 'BDI': 0.3365524258118314,
 'KHM': 0.9534774992301595,
 'CMR': 0.9313779027375948,
 'CAN': 0.8986601616756233,
 'CPV': 0.8185009818463631,
 'CYM': 1.066967233727

In [29]:
non_qual = []
for i in Countries:
    if abs(ratio_con[i] - 1) > 0.2:
        non_qual.append(i)

In [30]:
ratio_con["USA"]

0.7799760072870254

In [31]:
len(non_qual)

87

In [32]:
# Check calculation of import and export
t = T_df1.to_numpy()
fd = FD_df1.to_numpy()
print ((np.sum(t[:26, 26:]) + np.sum(fd[:26, 6:]))/ (np.sum(t[26:, :26]) + np.sum(fd[26:, :6])))

0.4415025814263115


In [33]:
ratio_con["AFG"]

0.44150258142631177

- M = M_0; % Margin 0 basic price
- M_2; % trade margin
- M_3; % transport margin 
- xout = sum(M,2); % Row-wise sums
- M_all = M_0 + M_2 + M_3;
- xin = sum(M_all,1);   %column sums
- xout(1) / xin(1)   % this should be 1.0 (perfect balance)
- A transport margin consists of those transport charges paid separately by the purchaser in taking delivery of the goods at the required time and place.


## Leontief Inverse

In [34]:
T_df2 = copy.copy(T_df)
Transaction_matrix = T_df2.to_numpy()
gross_output = list(Export["Gross_output"])

In [35]:
Transaction_matrix

array([[9.81350e+04, 2.53047e+01, 3.59372e+02, ..., 1.00000e-07,
        1.00000e-07, 3.98916e+05],
       [1.32408e+02, 4.18240e+02, 2.32455e+00, ..., 1.00000e-07,
        1.00000e-07, 2.86192e+04],
       [1.60391e+03, 3.77911e+01, 1.02990e+04, ..., 1.00000e-07,
        1.00000e-07, 3.76473e+04],
       ...,
       [3.30335e+00, 3.58883e+00, 4.79122e+00, ..., 1.01460e-01,
        5.17277e-01, 1.45728e+05],
       [5.94325e+00, 6.83916e+00, 8.99569e+00, ..., 5.98614e-01,
        1.01460e-01, 1.79240e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [36]:
# Small example
a = np.array([[1000,2000,3000],[4000,5000,6000],[7000,8000,9000]])
b = [10.0,100.0,1000.0]
for i in range(3):
    a[:, i] = a[:, i] / b[i]
print (a)

[[100  20   3]
 [400  50   6]
 [700  80   9]]


In [37]:
np.identity(3) - a

array([[ -99.,  -20.,   -3.],
       [-400.,  -49.,   -6.],
       [-700.,  -80.,   -8.]])

In [38]:
# A
A = np.zeros(shape=(4915,4915))
for i in range(4915):
    if gross_output[i] == 0:
        A[:, i] = 0
        continue
    A[:, i] = Transaction_matrix[:, i] / gross_output[i] 

In [39]:
np.identity(4915)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [40]:
# (I-A)^-1
Leontif_Inverse = np.linalg.inv(np.matrix(np.identity(4915) - A))

##### Check whether (I-A)^-1 * final demand = gross output

In [41]:
final_demand = np.array(Export["TFD"])
Leontif_Inverse * np.matrix(final_demand).T

matrix([[4.52045826e+06],
        [1.84252038e+05],
        [2.79435991e+05],
        ...,
        [5.73152175e+05],
        [7.03107207e+05],
        [4.25188853e+08]])

In [42]:
calculated_output = (Leontif_Inverse * np.matrix(final_demand).T).T.tolist()[0]

In [43]:
diff = []
for i in range(len(calculated_output)):
    diff.append(calculated_output[i] - gross_output[i])

In [44]:
np.mean(diff)

1.4394759860700955e-07

------

## Check Whether those countries have data
- ANT: Netherlands Antilles (Netherlands)
- PSE: Gaza Strip
- USR: USSR

In [6]:
Countries.index('ANT'),Countries.index('PSE'),Countries.index('USR') # Position of those regions

(120, 127, 174)

In [12]:
Countries

['AFG',
 'ALB',
 'DZA',
 'AND',
 'AGO',
 'ATG',
 'ARG',
 'ARM',
 'ABW',
 'AUS',
 'AUT',
 'AZE',
 'BHS',
 'BHR',
 'BGD',
 'BRB',
 'BLR',
 'BEL',
 'BLZ',
 'BEN',
 'BMU',
 'BTN',
 'BOL',
 'BIH',
 'BWA',
 'BRA',
 'VGB',
 'BRN',
 'BGR',
 'BFA',
 'BDI',
 'KHM',
 'CMR',
 'CAN',
 'CPV',
 'CYM',
 'CAF',
 'TCD',
 'CHL',
 'CHN',
 'COL',
 'COG',
 'CRI',
 'HRV',
 'CUB',
 'CYP',
 'CZE',
 'CIV',
 'PRK',
 'COD',
 'DNK',
 'DJI',
 'DOM',
 'ECU',
 'EGY',
 'SLV',
 'ERI',
 'EST',
 'ETH',
 'FJI',
 'FIN',
 'FRA',
 'PYF',
 'GAB',
 'GMB',
 'GEO',
 'DEU',
 'GHA',
 'GRC',
 'GRL',
 'GTM',
 'GIN',
 'GUY',
 'HTI',
 'HND',
 'HKG',
 'HUN',
 'ISL',
 'IND',
 'IDN',
 'IRN',
 'IRQ',
 'IRL',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'KWT',
 'KGZ',
 'LAO',
 'LVA',
 'LBN',
 'LSO',
 'LBR',
 'LBY',
 'LIE',
 'LTU',
 'LUX',
 'MAC',
 'MDG',
 'MWI',
 'MYS',
 'MDV',
 'MLI',
 'MLT',
 'MRT',
 'MUS',
 'MEX',
 'MCO',
 'MNG',
 'MNE',
 'MAR',
 'MOZ',
 'MMR',
 'NAM',
 'NPL',
 'NLD',
 'ANT',
 'NCL',
 'NZL',
 'NIC',
 'NER',


In [46]:
Countries.index('PAK'),Countries.index('ISR')

(129, 83)

- Get rid of USSR since its value makes no sence.

In [47]:
T_df

,0,1,2,3,4,5,6,7,8,9,...,4905,4906,4907,4908,4909,4910,4911,4912,4913,4914
0,98135.00000,25.30470,359.37200,214505.00000,2674.45000,32478.10000,2813.46000,139.37000,186.41400,19.09510,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.430900e-07,1.000000e-07,1.000000e-07,398916.00
1,132.40800,418.24000,2.32455,28665.90000,4.19821,1.56355,35.82680,18.74070,2.47318,1.07425,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,28619.20
2,1603.91000,37.79110,10299.00000,667.63800,68.57620,698.94000,62054.80000,6174.97000,188.68600,265.54800,...,1.048770e-07,1.043120e-07,1.042980e-07,1.000000e-07,1.001720e-07,1.002060e-07,1.000000e-07,1.000000e-07,1.000000e-07,37647.30
3,40408.80000,468.59600,54.37240,161090.00000,1532.57000,1237.13000,4558.57000,28.90200,54.17510,13.63440,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,5095.01
4,1706.29000,219.67600,385.27000,1929.72000,50932.00000,4957.07000,6176.40000,919.84400,4939.74000,3683.98000,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,20594.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,3.01483,3.20941,4.24502,5.19153,5.24824,5.24130,5.16952,5.18477,5.10443,5.17863,...,1.097940e+00,7.491060e-01,7.833540e-01,3.013900e-01,4.209160e-01,1.014600e-01,3.243420e+01,2.166330e+01,2.714370e+01,367530.00
4911,1.17719,1.37551,1.84375,2.25750,2.27674,2.29281,2.26491,2.26818,2.24488,2.25501,...,9.593950e-02,1.035910e-01,9.461920e-02,1.078180e-01,9.560480e-02,9.890780e-02,1.014600e-01,5.840000e-01,4.603710e-01,40930.00
4912,3.30335,3.58883,4.79122,5.92975,5.93691,5.97308,5.90872,5.91886,5.85291,5.88518,...,7.730420e-02,8.454030e-02,7.691460e-02,8.448920e-02,7.893080e-02,7.932750e-02,7.433410e-01,1.014600e-01,5.172770e-01,145728.00
4913,5.94325,6.83916,8.99569,11.27200,11.27610,11.35070,11.24740,11.25540,11.12770,11.17790,...,8.463560e-02,9.186710e-02,8.350150e-02,9.163710e-02,8.236980e-02,8.578660e-02,1.438470e+00,5.986140e-01,1.014600e-01,179240.00


In [48]:
T_df.iloc[120*26:121*26,:] # USSR transaction matrix

,0,1,2,3,4,5,6,7,8,9,...,4905,4906,4907,4908,4909,4910,4911,4912,4913,4914
3120,9.605930e-08,9.833260e-08,8.737660e-08,8.734140e-08,8.740360e-08,8.742340e-08,8.742110e-08,8.744380e-08,8.742980e-08,8.740860e-08,...,8.733770e-08,8.733770e-08,8.733770e-08,8.733770e-08,8.733770e-08,8.733770e-08,8.733770e-08,8.733770e-08,8.733770e-08,13.44390
3121,1.392680e-08,1.425640e-08,1.266800e-08,1.266290e-08,1.267190e-08,1.267480e-08,1.267440e-08,1.267770e-08,1.267570e-08,1.267260e-08,...,1.266230e-08,1.266230e-08,1.266230e-08,1.266230e-08,1.266230e-08,1.266230e-08,1.266230e-08,1.266230e-08,1.266230e-08,1.94911
3122,3.308970e-07,3.385940e-07,3.001360e-07,3.000150e-07,3.002280e-07,3.002960e-07,3.002880e-07,3.003660e-07,3.003180e-07,3.002450e-07,...,3.000000e-07,3.000000e-07,3.000000e-07,3.000000e-07,3.000000e-07,3.000000e-07,3.000000e-07,3.000000e-07,3.000000e-07,70.87980
3123,2.194160e-07,2.245850e-07,1.998830e-07,1.998020e-07,1.999440e-07,1.999900e-07,1.999850e-07,2.000360e-07,2.000040e-07,1.999560e-07,...,1.997950e-07,1.997950e-07,1.997950e-07,1.997950e-07,1.997950e-07,1.997950e-07,1.997950e-07,1.997950e-07,1.997950e-07,0.00000
3124,8.888130e-08,9.092560e-08,8.039550e-08,8.036330e-08,8.042010e-08,8.043830e-08,8.043620e-08,8.045690e-08,8.044410e-08,8.042470e-08,...,8.035900e-08,8.035900e-08,8.035900e-08,8.035900e-08,8.035900e-08,8.035900e-08,8.035900e-08,8.035900e-08,8.035900e-08,259.53900
3125,2.218420e-08,2.269440e-08,2.006530e-08,2.005730e-08,2.007150e-08,2.007600e-08,2.007550e-08,2.008070e-08,2.007750e-08,2.007260e-08,...,2.005620e-08,2.005620e-08,2.005620e-08,2.005620e-08,2.005620e-08,2.005620e-08,2.005620e-08,2.005620e-08,2.005620e-08,65.18940
3126,2.365930e-07,2.421150e-07,2.146130e-07,2.145270e-07,2.146790e-07,2.147270e-07,2.147220e-07,2.147770e-07,2.147430e-07,2.146910e-07,...,2.145170e-07,2.145170e-07,2.145170e-07,2.145170e-07,2.145170e-07,2.145170e-07,2.145170e-07,2.145170e-07,2.145170e-07,24.01370
3127,1.301020e-07,1.331760e-07,1.183630e-07,1.183150e-07,1.183990e-07,1.184260e-07,1.184230e-07,1.184540e-07,1.184350e-07,1.184060e-07,...,1.183110e-07,1.183110e-07,1.183110e-07,1.183110e-07,1.183110e-07,1.183110e-07,1.183110e-07,1.183110e-07,1.183110e-07,0.00000
3128,1.669690e-07,1.709020e-07,1.527800e-07,1.527190e-07,1.528270e-07,1.528610e-07,1.528570e-07,1.528960e-07,1.528720e-07,1.528350e-07,...,1.527060e-07,1.527060e-07,1.527060e-07,1.527060e-07,1.527060e-07,1.527060e-07,1.527060e-07,1.527060e-07,1.527060e-07,22.50250
3129,2.270000e-07,2.321590e-07,2.001130e-07,2.000340e-07,2.001730e-07,2.002170e-07,2.002120e-07,2.002630e-07,2.002320e-07,2.001840e-07,...,2.000000e-07,2.000390e-07,2.000610e-07,2.000000e-07,2.000000e-07,2.000000e-07,2.000000e-07,2.000000e-07,2.000000e-07,1539.72000


In [49]:
FD_df.loc[127*26:128*26-1,:]

,0,1,2,3,4,5,6,7,8,9,...,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139
3302,7.67663,7.67663,12.96430,7.89459,1.313690,7.89459,3.770960,3.770960,6.948790,6.203370,...,1.90740,1.90095,0.084747,1.90095,0.0,0.0,0.0,0.0,0.0,0.0
3303,1.55923,1.55923,2.95092,1.70516,0.363738,1.70516,0.660809,0.660809,0.976738,0.864839,...,1.89568,1.25541,0.048297,1.25541,0.0,0.0,0.0,0.0,0.0,0.0
3304,2.05353,2.05353,4.14759,2.37783,0.423908,2.37783,0.939629,0.939629,1.644470,1.377060,...,1.81154,1.10292,0.049347,1.10292,0.0,0.0,0.0,0.0,0.0,0.0
3305,4.22715,4.22715,8.27340,5.02009,0.865694,5.02009,1.819580,1.819580,3.606470,3.233920,...,1.72091,1.48290,0.075753,1.48290,0.0,0.0,0.0,0.0,0.0,0.0
3306,3.62573,3.62573,6.88547,4.14578,0.741164,4.14578,1.522860,1.522860,3.143860,2.520500,...,1.79934,1.27642,0.066808,1.27642,0.0,0.0,0.0,0.0,0.0,0.0
3307,1.74352,1.74352,3.52597,1.97264,0.360370,1.97264,0.724767,0.724767,1.247630,1.070790,...,1.74202,1.09659,0.044541,1.09659,0.0,0.0,0.0,0.0,0.0,0.0
3308,4.13193,4.13193,7.95304,4.77156,0.830333,4.77156,1.637310,1.637310,3.510280,2.923430,...,1.69425,1.47201,0.075351,1.47201,0.0,0.0,0.0,0.0,0.0,0.0
3309,1.99727,1.99727,3.88712,2.11359,0.395307,2.11359,0.865749,0.865749,1.665530,1.308550,...,1.75666,1.10320,0.044659,1.10320,0.0,0.0,0.0,0.0,0.0,0.0
3310,2.29971,2.29971,4.51742,2.49984,0.464641,2.49984,0.884853,0.884853,1.848140,1.480320,...,1.61060,1.06574,0.043572,1.06574,0.0,0.0,0.0,0.0,0.0,0.0
3311,1.08652,1.08652,2.08932,1.16916,0.257161,1.16916,0.329812,0.329812,0.570011,0.456865,...,1.70464,1.07523,0.043896,1.07523,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
FD_df.loc[120*26:121*26-1,:]

,0,1,2,3,4,5,6,7,8,9,...,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139
3120,8.734060e-08,8.734060e-08,8.733770e-08,8.733770e-08,9.426330e-08,8.733770e-08,9.429850e-08,9.429850e-08,9.483850e-08,9.404340e-08,...,1.767720e-06,7.658480e-07,0.000074,7.658480e-07,0.0,0.0,0.0,0.0,0.0,0.0
3121,1.266280e-08,1.266280e-08,1.266230e-08,1.266230e-08,1.366640e-08,1.266230e-08,1.367150e-08,1.367150e-08,1.374980e-08,1.363450e-08,...,2.562860e-07,1.110340e-07,0.000011,1.110340e-07,0.0,0.0,0.0,0.0,0.0,0.0
3122,3.000120e-07,3.000120e-07,3.000000e-07,3.000000e-07,3.235770e-07,3.000000e-07,3.238500e-07,3.238500e-07,3.257780e-07,3.234550e-07,...,6.610980e-06,3.026270e-06,0.000256,3.026270e-06,0.0,0.0,0.0,0.0,0.0,0.0
3123,1.998010e-07,1.998010e-07,1.997950e-07,1.997950e-07,2.155410e-07,1.997950e-07,2.155710e-07,2.155710e-07,2.168860e-07,2.151410e-07,...,4.409150e-06,2.812830e-06,0.000232,2.812830e-06,0.0,0.0,0.0,0.0,0.0,0.0
3124,8.036240e-08,8.036240e-08,8.035900e-08,8.035900e-08,8.665300e-08,8.035900e-08,8.690440e-08,8.690440e-08,8.726670e-08,8.675240e-08,...,2.735130e-06,1.696470e-06,0.000103,1.696470e-06,0.0,0.0,0.0,0.0,0.0,0.0
3125,2.005710e-08,2.005710e-08,2.005620e-08,2.005620e-08,2.162710e-08,2.005620e-08,2.169020e-08,2.169020e-08,2.178030e-08,2.165230e-08,...,6.839690e-07,4.236780e-07,0.000026,4.236780e-07,0.0,0.0,0.0,0.0,0.0,0.0
3126,2.145250e-07,2.145250e-07,2.145170e-07,2.145170e-07,2.313840e-07,2.145170e-07,2.315380e-07,2.315380e-07,2.330360e-07,2.312690e-07,...,8.510600e-06,6.774180e-06,0.000247,6.774180e-06,0.0,0.0,0.0,0.0,0.0,0.0
3127,1.183140e-07,1.183140e-07,1.183110e-07,1.183110e-07,1.276480e-07,1.183110e-07,1.276900e-07,1.276900e-07,1.284480e-07,1.274230e-07,...,2.416600e-06,1.094470e-06,0.000107,1.094470e-06,0.0,0.0,0.0,0.0,0.0,0.0
3128,1.527170e-07,1.527170e-07,1.527080e-07,1.527060e-07,1.647330e-07,1.527060e-07,1.648750e-07,1.648750e-07,1.658410e-07,1.645200e-07,...,3.101990e-06,1.490810e-06,0.000132,1.490810e-06,0.0,0.0,0.0,0.0,0.0,0.0
3129,2.000270e-07,2.000270e-07,2.000130e-07,2.000000e-07,2.156590e-07,2.000000e-07,2.162640e-07,2.162640e-07,2.175250e-07,2.164280e-07,...,9.665510e-06,5.402240e-06,0.000289,5.402240e-06,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
Tol_export['ANT'],Tol_export['NLD'],Tol_export['USR']

(2389.3811785602566, 618431055.4006425, 2565.236963075036)

In [52]:
# Countries = ["AFG", "ALB", "DZA", "AND", "AGO", "ATG", "ARG", "ARM", "ABW", "AUS", "AUT", "AZE", "BHS", "BHR", 
#              "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BMU", "BTN", "BOL", "BIH", "BWA", "BRA", "VGB", "BRN", 
#              "BGR", "BFA", "BDI", "KHM", "CMR", "CAN", "CPV", "CYM", "CAF", "TCD", "CHL", "CHN", "COL", "COG", 
#              "CRI", "HRV", "CUB", "CYP", "CZE", "CIV", "PRK", "COD", "DNK", "DJI", "DOM", "ECU", "EGY", "SLV", 
#              "ERI", "EST", "ETH", "FJI", "FIN", "FRA", "PYF", "GAB", "GMB", "GEO", "DEU", "GHA", "GRC", "GRL", 
#              "GTM", "GIN", "GUY", "HTI", "HND", "HKG", "HUN", "ISL", "IND", "IDN", "IRN", "IRQ", "IRL", "ISR", 
#              "ITA", "JAM", "JPN", "JOR", "KAZ", "KEN", "KWT", "KGZ", "LAO", "LVA", "LBN", "LSO", "LBR", "LBY", 
#              "LIE", "LTU", "LUX", "MAC", "MDG", "MWI", "MYS", "MDV", "MLI", "MLT", "MRT", "MUS", "MEX", "MCO", 
#              "MNG", "MNE", "MAR", "MOZ", "MMR", "NAM", "NPL", "NLD", "ANT", "NCL", "NZL", "NIC", "NER", "NGA", 
#              "NOR", "PSE", "OMN", "PAK", "PAN", "PNG", "PRY", "PER", "PHL", "POL", "PRT", "QAT", "KOR", "MDA", 
#              "ROU", "RUS", "RWA", "WSM", "SMR", "STP", "SAU", "SEN", "SRB", "SYC", "SLE", "SGP", "SVK", "SVN", 
#              "SOM", "ZAF", "SDS", "ESP", "LKA", "SUD", "SUR", "SWZ", "SWE", "CHE", "SYR", "TWN", "TJK", "THA", 
#              "MKD", "TGO", "TTO", "TUN", "TUR", "TKM", "USR", "UGA", "UKR", "ARE", "GBR", "TZA", "USA", "URY", 
#              "UZB", "VUT", "VEN", "VNM", "YEM", "ZMB", "ZWE", "ROW"]